In [4]:
import BHU
from BHU.KerasTransformers import preprocess_bucketize_col, preprocess_min_max_cols, preprocess_standard_scaler_cols
from BHU.KerasTransformers import ToDataFrame
import os
import numpy as np
import keys as k

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

keys = k.getKeys()

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

user_home : dict = BHU.get_UserHome(keys['SampleHouse'].strip("\'"))
user_home_details = BHU.get_PropertyDetail(user_home.get('property_id'))
hoi = BHU.get_HousesOfInterest(user_home, n=2000, listed_to_sold_ratio=0.3, verbose=True)
gd = BHU.GeoData(hoi['geo'])

fg = BHU.FeatureGenerator(
    houses = hoi['houses'],
    gd=gd,
    user_home=user_home_details
)

Loading SEATTLE_WA model.


In [5]:
# This is the whole data set
target_transformer = StandardScaler()
train_targets = target_transformer.fit_transform(np.array(fg.targets).reshape(-1,1))
train_features = fg.features

normalize_cols = ['lot_sqft', 'sqft']
bucketize_cols = ['year_built', 'distance_to_home', 'lat_winz', 'long_winz']
walk_score = ['walk_score']
dummy_cols = ['baths_full', 'baths_3qtr', 'baths_half', 'baths_1qtr', 'garage', 'stories', 'beds']

preprocess_data = ColumnTransformer(
    [
        ('normalize', StandardScaler(), normalize_cols),
        ('bucketize', preprocess_bucketize_col, bucketize_cols),
        ('dummy', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), dummy_cols),
        ('walkscore_mm', preprocess_min_max_cols, walk_score),
        ('walkscore_ss', preprocess_standard_scaler_cols, walk_score)
    ]
)

keras_pipeline = Pipeline(
    [
        ('to_data_frame', ToDataFrame()),
        ('preprocess', preprocess_data),
        ('keras_model', BHU.KerasModel(user_home, target_transformer))
    ]
)

keras_pipeline.set_params(**{
    'keras_model__load_model_if_available' : False,
    'keras_model__update_model' : False,
    'keras_model__save_model' : True,
    'preprocess__bucketize__bucketize__n_bins' : 20
})

keras_pipeline.fit(train_features, train_targets)

Epoch 1/100
38/38 [==============================] - 1s 3ms/step - loss: 0.6274 - mean_squared_error: 0.6274 - mean_absolute_error: 0.5473
Epoch 2/100
38/38 [==============================] - 0s 3ms/step - loss: 0.2760 - mean_squared_error: 0.2760 - mean_absolute_error: 0.3511
Epoch 3/100
38/38 [==============================] - 0s 3ms/step - loss: 0.2236 - mean_squared_error: 0.2236 - mean_absolute_error: 0.3219
Epoch 4/100
38/38 [==============================] - 0s 2ms/step - loss: 0.1967 - mean_squared_error: 0.1967 - mean_absolute_error: 0.3062
Epoch 5/100
38/38 [==============================] - 0s 2ms/step - loss: 0.1659 - mean_squared_error: 0.1659 - mean_absolute_error: 0.2826
Epoch 6/100
38/38 [==============================] - 0s 2ms/step - loss: 0.1523 - mean_squared_error: 0.1523 - mean_absolute_error: 0.2703
Epoch 7/100
38/38 [==============================] - 0s 2ms/step - loss: 0.1474 - mean_squared_error: 0.1474 - mean_absolute_error: 0.2682
Epoch 8/100
38/38 [========

INFO:tensorflow:Assets written to: ram://fc46b231354640f290bee911e9e3cc33/assets


INFO:tensorflow:Assets written to: ram://fc46b231354640f290bee911e9e3cc33/assets


Pipeline(steps=[('to_data_frame', ToDataFrame()),
                ('preprocess',
                 ColumnTransformer(transformers=[('normalize', StandardScaler(),
                                                  ['lot_sqft', 'sqft']),
                                                 ('bucketize',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('bucketize',
                                                                   KBinsDiscretizer(n_bins=20,
                                                                                    strategy='uniform'))]),
                                                  ['year_built',
                                                   'distance_to_home',
                                                   'lat_winz', 'long_winz']),
                                                 ('dummy',
                                                  OneHotEncoder(handl...
                                       'area_type': 'address',
                                       'centroid': {'lat': 47.572613,
                                                    'lon': -122.306358},
                                       'city': 'Seattle', 'country': 'USA',
                                       'full_address': ['3416 19th Ave S '
                                                        'Seattle WA 98144',
                                                        '3416 19th Ave S '
                                                        'Seattle WA'],
                                       'line': '3416 19th Ave S',
                                       'mpr_id': '1897471296',
                                       'postal_code': '98144',
                                       'prop_status': ['recently_sold',
                                                       'off_market'],
                                       'property_id': '1897471296',
                                       'state_code': 'WA',
                                       'validation_code': ['121']}))])

In [6]:
user_pred = keras_pipeline.predict(fg.user_features)
user_pred[0][0], fg.user_target

1/1 [==============================] - 0s 66ms/step


(686129.44, 972500)

In [62]:
#user_home_details['data']['property_detail']['price_history']
#user_home_details['data']['property_detail']['forwarded_mpr_ids']
#isinstance(user_home_details['data']['property_detail']['forwarded_mpr_ids'][0], str)

In [16]:
from BHU.API_Calls import get_PropertyValue
tt = get_PropertyValue('1897471296')

In [65]:
tt['data']['forecast_values']

[{'estimates': [{'estimate': 948878, 'date': '2023-02-01'},
   {'estimate': 951817, 'date': '2023-03-01'},
   {'estimate': 957236, 'date': '2023-04-01'},
   {'estimate': 964549, 'date': '2023-05-01'},
   {'estimate': 973404, 'date': '2023-06-01'},
   {'estimate': 980898, 'date': '2023-07-01'},
   {'estimate': 987120, 'date': '2023-08-01'},
   {'estimate': 992085, 'date': '2023-09-01'},
   {'estimate': 996012, 'date': '2023-10-01'},
   {'estimate': 999444, 'date': '2023-11-01'}],
  'source': {'type': 'corelogic', 'name': 'CoreLogic®'}},
 {'estimates': [{'estimate': 903444, 'date': '2023-02-01'},
   {'estimate': 903181, 'date': '2023-03-01'},
   {'estimate': 903451, 'date': '2023-04-01'},
   {'estimate': 903995, 'date': '2023-05-01'},
   {'estimate': 904681, 'date': '2023-06-01'},
   {'estimate': 905434, 'date': '2023-07-01'},
   {'estimate': 906215, 'date': '2023-08-01'},
   {'estimate': 907004, 'date': '2023-09-01'},
   {'estimate': 907789, 'date': '2023-10-01'},
   {'estimate': 908566

In [6]:
xx = fg.user_features.copy()
xx['baths_half'] += 1

new_worth = keras_pipeline.predict(xx)
new_worth

1/1 [==============================] - 0s 25ms/step


array([[1109256.5]], dtype=float32)

In [14]:
worth_change_ratio = new_worth / user_pred
worth_change_ratio

array([[1.0274155]], dtype=float32)

In [15]:
new_worth_to_user = worth_change_ratio * fg.user_target
new_worth_to_user

array([[999161.58735752]])

In [16]:
worth_delta = new_worth_to_user - fg.user_target
worth_delta

array([[26661.58735752]])

In [3]:
import pickle
fn = f'BHU/Saved Results/KerasModel/Seattle_WA.pkl'

with open(fn, 'rb') as f:
    xx = pickle.load(f)

2023-03-10 17:06:46.725418: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://3334748c73d14f38a219b73400871d90: INVALID_ARGUMENT: ram://3334748c73d14f38a219b73400871d90 is a directory.


In [7]:
if True and not False:
    print('hi')

hi


In [11]:
print(True)

True


In [6]:
wsl = [x.walk_score for x in fg.houses]